<h1>기사 크롤링으로 가져오기 & 파일로 저장</h1>

In [23]:
##gpt추천인데 제대로 작동 안하노 ;;가 아니고 제대로 작동하노!!!!!
import requests
from bs4 import BeautifulSoup

headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"}

url = "https://n.news.naver.com/article/009/0005078872?sid=102"

res = requests.get(url, headers=headers)
soup = BeautifulSoup(res.content, "html.parser")

# 본문 크롤링
date_time = soup.select_one(".media_end_head_info_datestamp_time")
reporter = soup.select_one(".media_end_head_journalist_name")
article = soup.select_one("#dic_area")

reporter = reporter.text[:3]
date_time = date_time.text[:10]
article = article.text
data = [(reporter, date_time, article)]
data
    

[('한상헌',
  '2023.01.25',
  '\n이투스, 지난해 수능 탐구 영역 분석사탐 높은 난도 탓에 과탐 수험생들인문계열에 지원할 경우 ‘부담’“탐구영역 단순암기는 고득점 어려워”\n\n\n\n 2022 2023학년도 사회탐구 만점 비율과 1∙2등급 원점수 누적 비율 비교<자료=이투스 교육평가연구소>이번 2023학년도 수능에서는 사회탐구의 변별력이 높아져 ‘탐구영역’이 정시 지원의 변수로 작용한 것으로 나타났다.25일 이투스 교육평가연구소에 따르면 이번 수능에선 국어·수학 못지않게 사회탐구의 난이도도 높았던 것으로 파악됐다.이번 수능에서 경제, 생활과 윤리, 윤리와 사상, 정치와 법, 사회문화 등의 과목의 만점자 비율은 1%가 되지 않았다.2022학년도 수능에서는 사탐 9개 과목 중 6개 과목의 1등급컷이 만점인 50점이었고, 원점수로 만점을 받았음에도 불구하고 백분위 점수로 환산했을 때 97점을 받은 과목은 세계지리 등 5과목이었다.이에 따라 한 문제라도 틀리게 되면 백분위 점수가 크게 하락해 정시 지원시 많은 어려움이 있었다.2023학년도 수능에서는 대부분 과목에서 만점자 비율이 대폭 하락하며 구조 자체가 변했다. 1등급컷이 만점인 과목도 6개에서 동아시아사 1개 과목으로 줄었다.1~2문제 차이로 등급·백분위가 하락하거나, 백분위 중간이 사라지는 현상도 발생하지 않아 밀집된 구조를 보였다.특히 사회문화의 경우 만점부터 47점까지는 백분위 기준으로 100점, 46점과 45점은 백분위 기준으로 99점으로 나와 사탐 영역에서는 좀처럼 볼 수 없는 사례가 나오기도 했다.이 때문에 2022학년도와 달리 정시 지원에서 과학탐구 응시자들이 인문계열에 지원할 때 유리함이 줄어들어 심적 부담이 크게 작용했을 것으로 판단된다.\n\n\n\n 2022 2023학년도 과학탐구 만점 비율과 1∙2등급 원점수 누적 비율 비교<자료=이투스 교육평가연구소>2023학년도 수능 과학탐구는 전년도와 비교해 화학Ⅰ의 체감 난이도 차이가 컸다.만점자 비율은 크게 차이가 나지 않았지만

In [24]:
import pandas as pd

col = ["작성자","날짜","본문내용"]

data_frame=pd.DataFrame(data,columns=col)
data_frame.to_csv("news_crawling.csv",mode='w',index = False)

<h1>데이터 불러오기</h1>

In [25]:
#데이터 불러오기
dataset = pd.read_csv("news_crawling.csv")
content = dataset["본문내용"]

content = content[0]
# print(content)


<h1>불러온 데이터 요약하기</h1>

In [26]:
import nltk
nltk.download('punkt')
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained('eenzeenee/t5-base-korean-summarization')
tokenizer = AutoTokenizer.from_pretrained('eenzeenee/t5-base-korean-summarization')

prefix = "summarize: "
sample = content

inputs = [prefix + sample]


inputs = tokenizer(inputs, max_length=512, truncation=True, return_tensors="pt")
output = model.generate(**inputs, num_beams=3, do_sample=True, min_length=10, max_length=100)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
result = nltk.sent_tokenize(decoded_output.strip())[0]

print('RESULT >>', result)

[nltk_data] Downloading package punkt to /Users/sudong/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


RESULT >> 2023학년도 수능에서는 사회탐구의 변별력이 높아져 탐구영역이 정시 지원의 변수로 작용했다.
